## Inicialização do Navegador

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


nav = webdriver.Chrome()

#importar / visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,2000,4500


## Definição da funções de busca do Google Shopping e Buscapé

In [3]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    # analisar se ele não tem nenhum termo banido
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome.lower():
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_todos_os_termos_produto(lista_termos_nome_produto, nome):
    # analisar se TEM TODOS os termos do nome do produto
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome.lower():
            tem_todos_termos_produtos = False
    return tem_todos_termos_produtos


def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    lista_ofertas = []
    
    # entrar no google
    nav.get("https://www.google.com.br/?hl=pt-BR")
    nav.find_element('xpath','//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)

    nav.find_element(By.PARTIAL_LINK_TEXT, 'Shopping').click()
    WebDriverWait(nav, 10).until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, 'Shopping')))
    
    # lista de informações do produto
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'KZmu8e' )
    
    # Esperar até que os resultados de pesquisa estejam visíveis
    WebDriverWait(nav, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'KZmu8e')))
    
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'ljqwrc').text
        nome = nome.lower()
        # analisar se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        # analisar se TEM TODOS os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_os_termos_produto(lista_termos_nome_produto, nome)

        # selecionar só os elementos tem_termos_banidos = False e os tem_todos_termos_produtos = True

        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = resultado.find_element(By.CLASS_NAME, 'T14wmb').text 
            preco = preco.replace("R$", "").replace(" ", "").replace("Usado", "").replace("Recondicionado", "")
            preco = preco.replace(",", ".")
            preco = preco.replace("..", ".")
            pontos = preco.count(".")
            if pontos > 1:
                preco = preco.replace(".", "", pontos - 1)
            preco = float(preco)
            
            # se o preco está entre o preco_minimo e preco_maximo
            if preco_minimo <= preco <= preco_maximo:   

                elemento_referencia = resultado.find_element(By.CLASS_NAME, 'ROMz4c')
                elemento_pai = elemento_referencia.find_element(By.XPATH, '..')
                link = elemento_pai.get_attribute('href')
                lista_ofertas.append((nome, preco, link))
                
    return lista_ofertas

def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    lista_ofertas = []
    
    # entrar no buscapé
    nav.get("https://www.buscape.com.br/")
    nav.find_element(By.XPATH, 
                     '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    
    # lista de informações do produto
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'ProductCard_ProductCard_Inner__tsD4M')
    # Esperar até que os resultados de pesquisa estejam visíveis
    WebDriverWait(nav, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ProductCard_ProductCard_Inner__tsD4M')))
    
    
    for resultado in lista_resultados:
        preco_str = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
        nome = resultado.find_element(By.CLASS_NAME, 'ProductCard_ProductCard_Name__LT7hv').text
        link = resultado.get_attribute("href")
        
        nome = nome.lower()
        # analisar se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
        
        # analisar se TEM TODOS os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_os_termos_produto(lista_termos_nome_produto, nome)
        
        # se não tem termos banidos e tem todos os termos do produto
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text 
            preco = preco.replace("R$", "").replace(" ", "").replace("Usado", "").replace("Recondicionado", "")
            preco = preco.replace(",", ".")
            preco = preco.replace("..", ".")
            pontos = preco.count(".")
            if pontos > 1:
                preco = preco.replace(".", "", pontos - 1)
            preco = float(preco)
            
            # se o preco está entre o preco_minimo e preco_maximo
            if preco_minimo <= preco <= preco_maximo:   
                lista_ofertas.append((nome, preco, link))
                
    return lista_ofertas

# Construindo a Lista de Ofertas

In [4]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    # Pesquisar o produto
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]

    # Busca no Google Shopping
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=["produto", "preco", "link"])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])

    # Busca no Buscapé
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=["produto", "preco", "link"])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])

# Exibe a tabela_ofertas
display(tabela_ofertas)



,produto,preco,link
0,"iphone 12 apple 64gb branco tela 6,1 12mp ios,...",3299.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
1,"iphone 12 apple 64gb azul tela de 6,1, câmera ...",3299.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
2,iphone 12 64gb apple branco desbloqueado - mgj...,3324.05,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
3,celular apple iphone 12 white 64gb vitrine/sem...,3099.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
4,"iphone 12 apple 64gb azul tela de 6,1, câmera ...",3299.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
5,"iphone 12 apple 64gb azul tela de 6,1, câmera ...",3299.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
6,aparelho apple iphone 12 64gb preto vitrine/se...,3099.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
7,"iphone 12 apple 64gb preto tela 6,1 12mp ios, ...",3299.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
8,"iphone 12 apple 64gb azul tela de 6,1, câmera ...",3299.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
9,iphone 12 64gb apple preto desbloqueado - mgj5...,3324.05,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...


## Importando tabela para excel

In [6]:
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

## Enviando Email

In [ ]:
import win32com.client as win32

if len(tabela_ofertas) > 0:
    # Inicialize o Outlook
    outlook = win32.Dispatch("Outlook.Application")

    # Crie um novo e-mail
    mail = outlook.CreateItem(0)

    # Preencha os detalhes do e-mail
    mail.Subject = "Assunto do E-mail"
    mail.HTMLBody = f"""
    <p> Prezados, </p>
    <p> Encontramos alguns produtos em oferta dentro da faixa de preço desejada </p>
    {tabela_ofertas.to_html(index=False)}
    <p> Att, </p>
    
    """
    mail.To = "destinatario@example.com"  # Substitua pelo endereço de e-mail do destinatário

    # Anexe arquivos (opcional)
    # mail.Attachments.Add("C:\\caminho\\para\\seuarquivo.txt")

    # Envie o e-mail
    mail.Send()

    print("E-mail enviado com sucesso!")